In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
         .builder
         .appName("Fire Department Calls for Service")
         .config("spark.sql.warehouse.dir", "./data/5-spark-warehouse")
         .getOrCreate())

23/05/09 19:43:13 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/09 19:43:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/09 19:43:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from sodapy import Socrata

In [4]:
client = Socrata("data.sfgov.org", None)

In [5]:
results = client.get("nuek-vuh3", limit=25000)
type(results)

list

In [6]:
results

[{'call_number': '221210313',
  'unit_id': 'E36',
  'incident_number': '22054955',
  'call_type': 'Outside Fire',
  'call_date': '2022-05-01T00:00:00.000',
  'watch_date': '2022-04-30T00:00:00.000',
  'received_dttm': '2022-05-01T02:58:25.000',
  'entry_dttm': '2022-05-01T02:59:15.000',
  'dispatch_dttm': '2022-05-01T02:59:25.000',
  'response_dttm': '2022-05-01T03:01:06.000',
  'on_scene_dttm': '2022-05-01T03:02:27.000',
  'call_final_disposition': 'Fire',
  'available_dttm': '2022-05-01T03:05:00.000',
  'address': 'GOUGH ST/GROVE ST',
  'city': 'San Francisco',
  'zipcode_of_incident': '94102',
  'battalion': 'B02',
  'station_area': '36',
  'box': '3265',
  'original_priority': '3',
  'priority': '3',
  'final_priority': '3',
  'als_unit': True,
  'call_type_group': 'Fire',
  'number_of_alarms': '1',
  'unit_type': 'ENGINE',
  'unit_sequence_in_call_dispatch': '1',
  'fire_prevention_district': '2',
  'supervisor_district': '5',
  'neighborhoods_analysis_boundaries': 'Hayes Valley',

In [7]:
from pyspark.sql.types import *

fire_schema = StructType([StructField('call_number', StringType(), True),
                StructField('unit_id', StringType(), True),
                StructField('incident_number', StringType(), True),
                StructField('call_type', StringType(), True),
                StructField('call_date', StringType(), True),
                StructField('watch_date', StringType(), True),
                StructField('call_final_disposition', StringType(), True),
                StructField('available_dttm', StringType(), True),
                StructField('address', StringType(), True),
                StructField('city', StringType(), True),
                StructField('zipcode_of_incident', StringType(), True),
                StructField('battalion', StringType(), True),
                StructField('station_area', StringType(), True),
                StructField('box', StringType(), True),
                StructField('original_priority', StringType(), True),
                StructField('priority', StringType(), True),
                StructField('final_priority', StringType(), True),
                StructField('als_unit', BooleanType(), True),
                StructField('call_type_group', StringType(), True),
                StructField('number_of_alarms', StringType(), True),
                StructField('unit_type', StringType(), True),
                StructField('unit_sequence_in_call_dispatch', StringType(), True),
                StructField('fire_prevention_district', StringType(), True),
                StructField('supervisor_district', StringType(), True),
                StructField('neighborhoods_analysis_boundaries', StringType(), True),
                StructField('case_location', StructType([
                    StructField('type', StringType(), True),
                    StructField('coordinates', ArrayType(FloatType()), True),
                                                            ]), True),
                StructField('rowid', StringType(), True)])

In [8]:
fireDF = spark.createDataFrame(results, fire_schema)

In [9]:
fireDF.show(6, False)

23/05/09 19:43:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/09 19:43:27 WARN TaskSetManager: Stage 0 contains a task of very large size (1946 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------+---------------+------------+-----------------------+-----------------------+----------------------+-----------------------+---------------------------+-------------+-------------------+---------+------------+----+-----------------+--------+--------------+--------+---------------+----------------+---------+------------------------------+------------------------+-------------------+---------------------------------+---------------------------------+-------------+
|call_number|unit_id|incident_number|call_type   |call_date              |watch_date             |call_final_disposition|available_dttm         |address                    |city         |zipcode_of_incident|battalion|station_area|box |original_priority|priority|final_priority|als_unit|call_type_group|number_of_alarms|unit_type|unit_sequence_in_call_dispatch|fire_prevention_district|supervisor_district|neighborhoods_analysis_boundaries|case_location                    |rowid        |
+-----------+-------+---

In [10]:
fireDF.printSchema()

root
 |-- call_number: string (nullable = true)
 |-- unit_id: string (nullable = true)
 |-- incident_number: string (nullable = true)
 |-- call_type: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- watch_date: string (nullable = true)
 |-- call_final_disposition: string (nullable = true)
 |-- available_dttm: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode_of_incident: string (nullable = true)
 |-- battalion: string (nullable = true)
 |-- station_area: string (nullable = true)
 |-- box: string (nullable = true)
 |-- original_priority: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- final_priority: string (nullable = true)
 |-- als_unit: boolean (nullable = true)
 |-- call_type_group: string (nullable = true)
 |-- number_of_alarms: string (nullable = true)
 |-- unit_type: string (nullable = true)
 |-- unit_sequence_in_call_dispatch: string (nullable = true)
 |-- fire_prevention_d

In [11]:
parquet_path = "./data/5-Fire_Calls"
fireDF.write.format("parquet").save(parquet_path)

23/05/09 19:43:29 WARN TaskSetManager: Stage 1 contains a task of very large size (1946 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
parquet_table = "Fire_Calls_Table"
fireDF.write.format("parquet").saveAsTable(parquet_table)

23/05/09 19:43:31 WARN TaskSetManager: Stage 2 contains a task of very large size (1946 KiB). The maximum recommended task size is 1000 KiB.
